# Modeling

### Library & Data Imports

In [1]:
import warnings; warnings.simplefilter('ignore')
import pandas as pd
import numpy as np

from sklearn.linear_model import LassoCV, RidgeCV, LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
plt.style.use('ggplot')

from bg_functions import *

In [2]:
data = pd.read_csv('final_data.csv', index_col=0)
data.head()

,name,year_published,min_players,max_players,playing_time,min_age,average_rating,average_weight,category_abstract_strategy,category_action_/_dexterity,...,mechanic_time_track,mechanic_trading,mechanic_trick-taking,mechanic_variable_phase_order,mechanic_variable_player_powers,mechanic_voting,mechanic_worker_placement,category_historical,category_culture,mechanic_movement
0,"Warhammer 40,000: Heroes of Black Reach",2018.0,2.0,2.0,120.0,14.0,7.63636,1.91252,0,0,...,0,0,0,0,0,0,0,0,0,1
1,The Russo-Georgian War of 2008,2010.0,1.0,2.0,90.0,12.0,7.22222,3.00000,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Frher oder Spter,2009.0,2.0,5.0,30.0,8.0,4.50000,1.00000,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Wings Over France,1992.0,1.0,1.0,60.0,10.0,7.50000,3.22220,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Mothra vs. Godzilla,1982.0,2.0,2.0,90.0,10.0,6.98143,1.50000,0,0,...,0,0,0,0,0,0,0,0,1,0


### Setup

In [3]:
y = data['average_rating']
X = data.drop(['name', 'average_rating'], axis=1)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

### Baseline Model

In [5]:
lr = LinearRegression()
lr.fit(X_train, y_train)
base_preds = lr.predict(X_test)
print('Base MSE:', mean_squared_error(y_test, base_preds))
print('Base R^2 Score:', r2_score(y_test, base_preds))

Base MSE: 1.5854491022049808
Base R^2 Score: 0.21685720352690874


In [6]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [7]:
lr.fit(X_train_scaled, y_train)
base_preds = lr.predict(X_test_scaled)
print('Base MSE:', mean_squared_error(y_test, base_preds))
print('Base R^2 Score:', r2_score(y_test, base_preds))

Base MSE: 1.5854491022049804
Base R^2 Score: 0.21685720352690896


### Model Improvement Testing

In [8]:
lasso = LassoCV(cv=5)
create_model(lasso, X_train_scaled, y_train, X_test_scaled, y_test)

LassoCV(alphas=None, copy_X=True, cv=5, eps=0.001, fit_intercept=True,
    max_iter=1000, n_alphas=100, n_jobs=None, normalize=False,
    positive=False, precompute='auto', random_state=None,
    selection='cyclic', tol=0.0001, verbose=False)
MSE: 1.5853336267080753
R2: 0.2169142433924237


In [9]:
ridge = RidgeCV()
create_model(ridge, X_train_scaled, y_train, X_test_scaled, y_test)

RidgeCV(alphas=array([ 0.1,  1. , 10. ]), cv=None, fit_intercept=True,
    gcv_mode=None, normalize=False, scoring=None, store_cv_values=False)
MSE: 1.5854458239079268
R2: 0.21685882286285518


In [10]:
forest = RandomForestRegressor(n_estimators=100, random_state=42)
create_model(forest, X_train_scaled, y_train, X_test_scaled, y_test)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False)
MSE: 1.3880139331356636
R2: 0.3143815770385171


In [11]:
gradient = GradientBoostingRegressor()
create_model(gradient, X_train_scaled, y_train, X_test_scaled, y_test)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)
MSE: 1.3397322786438657
R2: 0.3382306112739696


In [12]:
grid_forest = RandomForestRegressor()
forest_params = {
    'n_estimators': [10, 50, 100],
    'max_depth': [None, 2, 4, 5],
    'max_features': ['auto', 'sqrt', 'log2', None]
}
gridsearch_model(grid_forest, X_train_scaled, y_train, X_test_scaled,
                 y_test, forest_params, title='Random Forest Gridsearch')

Random Forest Gridsearch
BEST PARAMS:
{'max_depth': None, 'max_features': 'sqrt', 'n_estimators': 100}

Best R2 Score: 0.3381


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [18]:
grid_gradient = GradientBoostingRegressor()
gradient_params = {
    'learning_rate': [0.1, 0.5],
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 2, 5],
    'max_features': ['auto', 'sqrt', None]
}
gridsearch_model(grid_gradient, X_train_scaled, y_train, X_test_scaled,
                 y_test, gradient_params, title='Gradient Boosting Gridsearch',
                 verbose=5)

Fitting 3 folds for each of 54 candidates, totalling 162 fits
[CV] learning_rate=0.1, max_depth=None, max_features=auto, n_estimators=50 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  learning_rate=0.1, max_depth=None, max_features=auto, n_estimators=50, score=-0.21761966713421743, total=11.6min
[CV] learning_rate=0.1, max_depth=None, max_features=auto, n_estimators=50 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 11.6min remaining:    0.0s


[CV]  learning_rate=0.1, max_depth=None, max_features=auto, n_estimators=50, score=-0.21962561082336896, total=11.8min
[CV] learning_rate=0.1, max_depth=None, max_features=auto, n_estimators=50 


...

[CV]  learning_rate=0.5, max_depth=None, max_features=sqrt, n_estimators=50, score=0.12052678033854602, total=  51.9s
[CV] learning_rate=0.5, max_depth=None, max_features=sqrt, n_estimators=50 
[CV]  learning_rate=0.5, max_depth=None, max_features=sqrt, n_estimators=50, score=0.10480441635640125, total=  53.3s
[CV] learning_rate=0.5, max_depth=None, max_features=sqrt, n_estimators=100 
[CV]  learning_rate=0.5, max_depth=None, max_features=sqrt, n_estimators=100, score=0.11807877754914531, total= 1.2min
[CV] learning_rate=0.5, max_depth=None, max_features=sqrt, n_estimators=100 
[CV]  learning_rate=0.5, max_depth=None, max_features=sqrt, n_estimators=100, score=0.12520685218219763, total= 1.2min
[CV] learning_rate=0.5, max_depth=None, max_features=sqrt, n_estimators=100 
[CV]  learning_rate=0.5, max_depth=None, max_features=sqrt, n_estimators=100, score=0.11141290565748307, total= 1.1min
[CV] learning_rate=0.5, max_depth=None, max_features=sqrt, n_estimators=200 
[CV]  learning_rate=0.5

[CV]  learning_rate=0.5, max_depth=2, max_features=None, n_estimators=200, score=0.33821556835256805, total=  12.4s
[CV] learning_rate=0.5, max_depth=5, max_features=auto, n_estimators=50 
[CV]  learning_rate=0.5, max_depth=5, max_features=auto, n_estimators=50, score=0.3374802745215809, total=   9.4s
[CV] learning_rate=0.5, max_depth=5, max_features=auto, n_estimators=50 
[CV]  learning_rate=0.5, max_depth=5, max_features=auto, n_estimators=50, score=0.336477970788573, total=   9.3s
[CV] learning_rate=0.5, max_depth=5, max_features=auto, n_estimators=50 
[CV]  learning_rate=0.5, max_depth=5, max_features=auto, n_estimators=50, score=0.3225621700174135, total=   9.5s
[CV] learning_rate=0.5, max_depth=5, max_features=auto, n_estimators=100 
[CV]  learning_rate=0.5, max_depth=5, max_features=auto, n_estimators=100, score=0.32710083717795413, total=  19.1s
[CV] learning_rate=0.5, max_depth=5, max_features=auto, n_estimators=100 
[CV]  learning_rate=0.5, max_depth=5, max_features=auto, n_e

[Parallel(n_jobs=1)]: Done 162 out of 162 | elapsed: 446.2min finished


Gradient Boosting Gridsearch
BEST PARAMS:
{'learning_rate': 0.1, 'max_depth': 5, 'max_features': 'auto', 'n_estimators': 200}

Best R2 Score: 0.3566


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=5,
             max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=200,
             n_iter_no_change=None, presort='auto', random_state=None,
             subsample=1.0, tol=0.0001, validation_fraction=0.1, verbose=0,
             warm_start=False)